In [1]:
import re
import datetime
import pandas as pd
from osisoft.pidevclub.piwebapi.pi_web_api_client import PIWebApiClient
from osisoft.pidevclub.piwebapi.rest import ApiException

In [2]:
client = PIWebApiClient("https://piwebapi.cammesa.com/piwebapi",
                            useKerberos=False, verifySsl=True)

In [3]:
df_paths = pd.read_csv('paths_reducido.csv')
paths = df_paths['Paths'].tolist()


In [ ]:
df_paths = pd.DataFrame([paths], columns=paths)
df_paths = df_paths.iloc[1:]
df_paths.columns = df_paths.columns.str.replace('pi:\\\CAMHIS01\\', '')

df_paths.head()

In [4]:
webs_id = []
for path in paths:
    try:
        point = client.point.get_by_path(path[3:])
        webs_id.append(point.web_id)
    except Exception as e:
        # Handle exceptions if needed
        print(f"Error retrieving information for path {path}: {e}")

In [5]:
current_values = []
for web_id in webs_id:
    current_values.append(client.stream.get_value(web_id=web_id).value)

time = client.stream.get_value(web_id=web_id).timestamp
print(current_values)

[1883.708, 28.1273766, 29.2504539, 266.985046, 55.1896667, -24.6382065, 176.799988, -32.21, -25.02, 131.269989, 17.0, 0.0, -3.9, 129.9, 34.4483643, 34.3773727, 0.0, 0.0, 188.695923, -44.7451019, -8.846033, -5.13546753, 0.0, 0.0, 0.0, 0.0, 13.6299992, 13.76, -39.6899872, 8.09001, 132.519989, 3.23001051, 0.0, 0.0, 0.0, 188.2309, 44.38347, 4.509226, 135.594971, 249.937408, 210.14856, 58.76924, -0.9996643, 34.2226868, 108.062988, 24.7126, -1.65957594, 129.896454, 330.3, -7.422, -2.21399975, 13.597, 64.70358, 13.5055885, 5.46754551, 130.374542, 50.0207748, 3259.69141, 55.6944427, 29.005188, 11.1229324, 33.1, 7.27999973, 3.46, 49.98726, 22.490715, -0.0599999949, -0.42899996, 0.0, 0.0, 12.2897015, -0.873216033, -8.119999, -3.08999968, 34.6, 145.6, -26.3999977, -22.5199966, 136.199982, 142.210083, 120.985184, 22.6984634, -4.99832153, 32.0, 6.58, 2.37, 124.349991, 26.4064484, 5.68314648, 1.51592445, 170.4844, 71.8282, 33.75795, 25.0678978, 6.316351, 2.46328974, 53.8254929, 11.1459351, 4.377537,

In [23]:
df_current_values = pd.DataFrame([current_values], columns=paths)
df_time = pd.DataFrame([time], columns=['Timestamp']) 
df_current_values.columns = df_current_values.columns.str.replace('pi:\\\CAMHIS01\\', '')

df_final = pd.concat([df_time, df_current_values], axis = 1)

df_final.head()

,Timestamp,DEM_NEA/TOT_DEM_____P.AV,RESISTEN/TEMP.AV,RINCON/TEMP.AV,A.BROWN/132/ROCA1/I.AV,A.BROWN/132/ROCA1/P.AV,A.BROWN/132/ROCA1/Q.AV,ALLEN/132/G.ROCA/I.AV,ALLEN/132/G.ROCA/P.AV,ALLEN/132/G.ROCA/Q.AV,...,S.ISIDRO/500/TRAFO2/V.AV,SGO.ESTE/13/ROCA/I.AV,SMARTNEA/13/TRAFO2/I.AV,SMARTNEA/13/TRAFO2/P.AV,SMARTNEA/13/TRAFO2/Q.AV,SMARTNEA/13/TRAFO2/V.AV,SMARTNEA/132/E.COLOR/I.AV,SMARTNEA/132/E.COLOR/P.AV,SMARTNEA/132/E.COLOR/Q.AV,SMARTNEA/132/E.COLOR/V.AV
0,2024-03-01T23:08:20Z,1883.708,28.127377,29.250454,266.985046,55.189667,-24.638206,176.799988,-32.21,-25.02,...,504.09552,202.695038,87.75,-1.6038,1.28304,13.6488,0.0,0.0,0.0,131.299988


In [24]:
df_final.set_index('Timestamp', inplace=True)

In [25]:
df_final_hora = df_final.copy()
df_final_hora.index = pd.to_datetime(df_final_hora.index).tz_convert('America/Argentina/Buenos_Aires').tz_localize(None)

df_final_hora

,DEM_NEA/TOT_DEM_____P.AV,RESISTEN/TEMP.AV,RINCON/TEMP.AV,A.BROWN/132/ROCA1/I.AV,A.BROWN/132/ROCA1/P.AV,A.BROWN/132/ROCA1/Q.AV,ALLEN/132/G.ROCA/I.AV,ALLEN/132/G.ROCA/P.AV,ALLEN/132/G.ROCA/Q.AV,ALLEN/132/G.ROCA/V.AV,...,S.ISIDRO/500/TRAFO2/V.AV,SGO.ESTE/13/ROCA/I.AV,SMARTNEA/13/TRAFO2/I.AV,SMARTNEA/13/TRAFO2/P.AV,SMARTNEA/13/TRAFO2/Q.AV,SMARTNEA/13/TRAFO2/V.AV,SMARTNEA/132/E.COLOR/I.AV,SMARTNEA/132/E.COLOR/P.AV,SMARTNEA/132/E.COLOR/Q.AV,SMARTNEA/132/E.COLOR/V.AV
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-03-01 20:08:20,1883.708,28.127377,29.250454,266.985046,55.189667,-24.638206,176.799988,-32.21,-25.02,131.269989,...,504.09552,202.695038,87.75,-1.6038,1.28304,13.6488,0.0,0.0,0.0,131.299988
